
1. data location: ![https://datahub.ruc.edu.cn/org/RUC/task/67bff956aa62251c80841535/67bff956aa62251c80841556/manage]

2. data analysis, data clean and data preprocessing (Please refer to Ch1 and Ch2 codes from our textbook and my slides)

3. delete the features with data leakage issues such as community price in the housing project(数据泄露问题，即训练集中存在与测试集相关联的信息，导致即使训练集上表现不错，实际效果上仍然一般)

4. Report both your: in-sample, out of sample and 6 folds cross validation model performance for your OLS, LASSO, Ridge and best model (please use Mean Absolute Error, RMAE to report the performance). Also, please report the total number of predictions after you remove the outlier of the sample.




Your score only depends on the contents and innovations mentioned by your presentations, but your presentation should be backed by your codes and results (!!!Your results should be checked during the presentation!!!)



Only Linear Algorithms in midterm (OLS, LASSO, Ridge and etc., but you can use fancy feature engineering)

max 3 slides (not counts on the frontpage)


# 一、整合数据

### 1、excel中初步处理，读入处理后文件

detail数据中：年份若为空，用2010补充；用水数据：商水/民水：1；供暖数据：集中供暖：1

In [390]:
import pandas as pd

In [392]:
Training=pd.read_excel(r'C:\Users\86188\Desktop\修改ruc_Class25Q1_train.xlsx')
Testing=pd.read_excel(r'C:\Users\86188\Desktop\修改ruc_Class25Q1_test.xlsx')

In [393]:
Training.info()
Testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84133 entries, 0 to 84132
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   编号      84133 non-null  int64 
 1   城市      84133 non-null  int64 
 2   区域      84133 non-null  int64 
 3   板块      84133 non-null  int64 
 4   环线      42726 non-null  object
 5   小区名称    84133 non-null  object
 6   价格      84133 non-null  int64 
 7   房屋户型    83528 non-null  object
 8   所在楼层    84133 non-null  object
 9   建筑面积    84133 non-null  object
 10  套内面积    25146 non-null  object
 11  房屋朝向    84133 non-null  object
 12  建筑结构    83528 non-null  object
 13  装修情况    83528 non-null  object
 14  配备电梯    84133 non-null  int64 
 15  别墅类型    749 non-null    object
 16  交易时间    84133 non-null  object
 17  交易权属    84133 non-null  object
 18  上次交易    55180 non-null  object
 19  房屋用途    84131 non-null  object
 20  房屋年限    54351 non-null  object
 21  产权所属    84133 non-null  int64 
 22  房屋优势    68069 non-null

In [394]:
Detail=pd.read_excel(r'C:\Users\86188\Desktop\ruc_Class25Q1_details.xlsx')
Rent=pd.read_excel(r'C:\Users\86188\Desktop\ruc_Class25Q1_rent.xlsx')

In [395]:
Rent.head()

,Unnamed: 0,城市,小区名称,价格,面积
0,0,0,东方雅苑,6021,37.36㎡
1,1,0,金地未未来,5099,55.42㎡
2,2,0,设计师广场,5038,35.44㎡
3,3,0,建材城东一里,6134,60.00㎡
4,4,0,融泽嘉园中路1号院,7909,86.76㎡


In [396]:
Detail.head()

,Unnamed: 0,城市,小区名称,建筑年代,绿化率,容积率,供水,供暖
0,0,2,三峡医专学苑新村,2010年,NaN,NaN,0,0
1,1,2,宁静苑,2010年,NaN,NaN,0,0
2,2,2,电信小区,2010年,NaN,NaN,0,0
3,3,0,三元街甲19号院,1970-1979年,30%,3.5,0,1
4,4,0,东交民巷11号院,1975-1984年,30%,2.0,0,1


### 2、预处理detail和rent文件

#### 绿化率去除百分号，用平均数填补空缺

In [404]:
Detail['绿化率'] = Detail['绿化率'].str.replace('%','').astype('float')/100

In [405]:
def fill_missing_by_city(df, column):
        city_mean = df.groupby('城市')[column].transform('mean')
        df[column] = df[column].fillna(city_mean)

In [408]:
fill_missing_by_city(Detail, '绿化率')

#### 容积率用平均数填补空缺

In [411]:
fill_missing_by_city(Detail, '容积率')

In [412]:
Detail.head()

,Unnamed: 0,城市,小区名称,建筑年代,绿化率,容积率,供水,供暖
0,0,2,三峡医专学苑新村,2010年,0.293323,3.284068,0,0
1,1,2,宁静苑,2010年,0.293323,3.284068,0,0
2,2,2,电信小区,2010年,0.293323,3.284068,0,0
3,3,0,三元街甲19号院,1970-1979年,0.300000,3.500000,0,1
4,4,0,东交民巷11号院,1975-1984年,0.300000,2.000000,0,1


#### 建筑年代转化为建筑建成时长

In [416]:
Detail['建筑年代'] = Detail['建筑年代'].str.replace('年','')

In [418]:
def parse_year(year_str):
    year_str = str(year_str).strip()
    if '-' in year_str:
        parts = year_str.split('-', 1)
        valid_parts = [x for x in parts if x.isdigit()]  # 过滤有效数字部分
        if len(valid_parts) >= 2:
            start, end = map(int, valid_parts[:2])
            return (start + end) / 2
        else:
            return np.nan  # 处理不完整范围（如"2020-")
    else:
        return int(year_str) if year_str.isdigit() else np.nan

# 生成新列
Detail['建成年限'] = Detail['建筑年代'].apply(parse_year)

In [420]:
Detail.head()

,Unnamed: 0,城市,小区名称,建筑年代,绿化率,容积率,供水,供暖,建成年限
0,0,2,三峡医专学苑新村,2010,0.293323,3.284068,0,0,2010.0
1,1,2,宁静苑,2010,0.293323,3.284068,0,0,2010.0
2,2,2,电信小区,2010,0.293323,3.284068,0,0,2010.0
3,3,0,三元街甲19号院,1970-1979,0.300000,3.500000,0,1,1974.5
4,4,0,东交民巷11号院,1975-1984,0.300000,2.000000,0,1,1979.5


#### rent数据用面积除价格得到每平价格

In [423]:
# 去除单位
Rent['面积'] = Rent['面积'].str.replace('㎡','').astype('float')
Rent['价格'].astype('float')
fill_missing_by_city(Rent, '面积')
fill_missing_by_city(Rent, '价格')

In [425]:
# 除法
Rent['每平租金'] = Rent['价格']/Rent['面积']

In [427]:
Rent.head()

,Unnamed: 0,城市,小区名称,价格,面积,每平租金
0,0,0,东方雅苑,6021,37.36,161.161670
1,1,0,金地未未来,5099,55.42,92.006496
2,2,0,设计师广场,5038,35.44,142.155756
3,3,0,建材城东一里,6134,60.00,102.233333
4,4,0,融泽嘉园中路1号院,7909,86.76,91.159521


In [429]:
# 计算小区平均租金

community_avg_rent = Rent.groupby('小区名称', as_index=False)['每平租金'].mean()

community_avg_rent.columns = ['小区名称', '平均每平租金(元)']


In [431]:
Cleanedrent = community_avg_rent
Cleanedrent.head()

,小区名称,平均每平租金(元)
0,10号名邸,110.488041
1,3532小区,31.736003
2,3号LOFT,57.727980
3,45所1区,26.091715
4,E动空间,38.134623


#### 保存

In [434]:
Cleanedrent.to_csv(r'D:\Python_Output\Cleanedrent.csv')
Rent.to_csv(r'D:\Python_Output\Rent.csv')
Detail.to_csv(r'D:\Python_Output\Detail.csv')

### 3、合并数据

In [437]:
cleaned_rent = pd.read_csv(r'D:\Python_Output\Cleanedrent.csv')
detail = pd.read_csv(r'D:\Python_Output\Detail.csv')

#### 将Rent合并到 Detail中

In [440]:
detail = detail.rename(columns={'名称': '小区名称'})

In [442]:
detail_merged = pd.merge(
    detail,
    cleaned_rent[['小区名称', '平均每平租金(元)']],
    on='小区名称',
    how='left'
)

# 检查合并结果
print("【Detail合并后缺失值统计】")
print(detail_merged['平均每平租金(元)'].isnull().sum())

【Detail合并后缺失值统计】
298


填补缺失值

In [445]:
fill_missing_by_city(detail_merged, '平均每平租金(元)')

#### 将 Detail合并到 Train和 Test中

In [448]:
print("【Detail表重复值统计】")
print(detail_merged.duplicated(subset=["小区名称"]).sum())

【Detail表重复值统计】
127


__注意此处一定要去除重复后再合并，否则会导致train和test生成新的列（由于一对多）__

In [451]:
detail_unique = detail_merged.drop_duplicates(subset=["小区名称"], keep="first")
print("【Detail表重复值统计】")
print(detail_unique.duplicated(subset=["小区名称"]).sum())

【Detail表重复值统计】
0


In [453]:
# 合并到Training
training_merged = pd.merge(
    training,
    detail_unique,
    left_on='小区名称',
    right_on='小区名称',
    how='left'
)

# 合并到Testing
testing_merged = pd.merge(
    testing,
    detail_unique,
    left_on='小区名称',
    right_on='小区名称',
    how='left'
)

# 检查最终结果
print("\n【Training合并后字段示例】")
print(training_merged.columns)
print("\n【Testing合并后字段示例】")
print(testing_merged.columns)


【Training合并后字段示例】
Index(['编号', '城市_x', '区域', '板块', '环线', '小区名称', '价格', '房屋户型', '所在楼层', '建筑面积',
       '套内面积', '房屋朝向', '建筑结构', '装修情况', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易',
       '房屋用途', '房屋年限', '产权所属', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', '年份',
       'Unnamed: 0.1', 'Unnamed: 0', '城市_y', '建筑年代', '绿化率', '容积率', '供水', '供暖',
       '建成年限', '平均每平租金(元)'],
      dtype='object')

【Testing合并后字段示例】
Index(['ID', '城市_x', '区域', '板块', '环线', '小区名称', '房屋户型', '所在楼层', '建筑面积', '套内面积',
       '房屋朝向', '建筑结构', '装修情况', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途',
       '房屋年限', '产权所属', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', '年份',
       'Unnamed: 0.1', 'Unnamed: 0', '城市_y', '建筑年代', '绿化率', '容积率', '供水', '供暖',
       '建成年限', '平均每平租金(元)'],
      dtype='object')


### 4、检查数据中的空缺

In [456]:
testing_merged.head()

,ID,城市_x,区域,板块,环线,小区名称,房屋户型,所在楼层,建筑面积,套内面积,...,Unnamed: 0.1,Unnamed: 0,城市_y,建筑年代,绿化率,容积率,供水,供暖,建成年限,平均每平租金(元)
0,0,0,45,416,四至五环,泛海容郡,3室2厅1厨3卫,中楼层 (共25层),209.2㎡,165.71㎡,...,1054.0,1054.0,0.0,2014-2019,0.350,2.0,0.0,1.0,2016.5,231.381974
1,1,0,45,414,四至五环,慧谷金色家园,4室2厅1厨2卫,低楼层 (共9层),163.69㎡,NaN,...,1037.0,1037.0,0.0,2000-2004,0.371,2.1,0.0,1.0,2002.0,122.896354
2,2,0,43,289,五至六环,天通苑中苑,2室2厅1厨1卫,中楼层 (共8层),102.92㎡,NaN,...,892.0,892.0,0.0,2007-2017,0.360,2.9,1.0,1.0,2012.0,61.878339
3,3,0,39,374,NaN,富乐小区南里,3室2厅1厨2卫,中楼层 (共6层),109.66㎡,NaN,...,782.0,782.0,0.0,1995-2000,0.300,2.0,0.0,1.0,1997.5,40.023846
4,4,0,79,724,二至三环,百万庄未区,2室1厅1厨1卫,底层 (共3层),57.2㎡,NaN,...,2453.0,2453.0,0.0,1950-1978,0.300,1.5,0.0,1.0,1964.0,99.790921


In [458]:
# testing_merged.drop(columns=['Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0','城市_y', '建筑年代',], inplace=True)
# training_merged.drop(columns=['Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0','城市_y', '建筑年代',], inplace=True)

testing_merged.drop(columns=['Unnamed: 0.1','Unnamed: 0','城市_y', '建筑年代',], inplace=True)
training_merged.drop(columns=['Unnamed: 0.1','Unnamed: 0','城市_y', '建筑年代',], inplace=True)

虚拟变量补为0

In [461]:
testing_merged['供水'] = testing_merged['供水'].fillna(0)
testing_merged['供暖'] =testing_merged['供暖'].fillna(0)
testing_merged = testing_merged.rename(columns={'城市_x': '城市'})

training_merged['供水'] = training_merged['供水'].fillna(0)
training_merged['供暖'] = training_merged['供暖'].fillna(0)
training_merged = training_merged.rename(columns={'城市_x': '城市'})

数值变量用同城市均值补齐

In [464]:
fill_missing_by_city(training_merged, '绿化率')
fill_missing_by_city(training_merged, '容积率')
fill_missing_by_city(training_merged, '建成年限')
fill_missing_by_city(training_merged, '平均每平租金(元)')

fill_missing_by_city(testing_merged, '绿化率')
fill_missing_by_city(testing_merged, '容积率')
fill_missing_by_city(testing_merged, '建成年限')
fill_missing_by_city(testing_merged, '平均每平租金(元)')

### 5、保存数据

In [467]:
testing_merged.to_csv(r'D:\Python_Output\TestmergedData.csv')
training_merged.to_csv(r'D:\Python_Output\TrainmergedData.csv')

# 二、清洗数据


### 1、读取初步清洗后数据

In [5]:
import pandas as pd
Traindata=pd.read_csv(r'D:\Python_Output\TrainmergedData.csv')
Testdata=pd.read_csv(r'D:\Python_Output\TestmergedData.csv')

### 2、处理面积数据：套内面积和建筑面积

In [8]:
Traindata['建筑面积'] = Traindata['建筑面积'].str.replace('㎡','').astype(float)
Traindata['套内面积'] = Traindata['套内面积'].str.replace('㎡','').astype(float)
Testdata['建筑面积'] = Testdata['建筑面积'].str.replace('㎡','').astype(float)
Testdata['套内面积'] = Testdata['套内面积'].str.replace('㎡','').astype(float)
lenTrain=len(Traindata['建筑面积'])
lenTest=len(Testdata['建筑面积'])


In [10]:
mask_train = Traindata['套内面积'].isna()
Traindata.loc[mask_train, '套内面积'] = 0.95 * Traindata['建筑面积']
Traindata.loc[~mask_train, '套内面积'] = 0.2*Traindata['建筑面积'] + 0.8*Traindata['套内面积']

# 测试集处理
mask_test = Testdata['套内面积'].isna()
Testdata.loc[mask_test, '套内面积'] = 0.95 * Testdata['建筑面积']
Testdata.loc[~mask_test, '套内面积'] = 0.2*Testdata['建筑面积'] + 0.8*Testdata['套内面积']


In [12]:
Traindata['建成年限'] = 2025-Traindata['建成年限']
Testdata['建成年限'] = 2025-Testdata['建成年限']

### 3、处理数值型虚拟变量

##### 用独热生成虚拟变量

In [16]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# 定义处理函数

def process_region(train_df, test_df=None, Target_column='区域'):
    # 填充缺失值（统一用0表示未知区域）
    train_df[Target_column] = train_df[Target_column].fillna(0).astype(int).astype(str)
    
    # 初始化编码器
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    encoder.fit(train_df[[Target_column]])
    
    # 生成列名（区域_编号）
    region_cols = [f"{Target_column}_{int(cat)}"  
                   for cat in encoder.categories_[0]]
    
    # 转换训练集
    train_encoded = pd.DataFrame(
        encoder.transform(train_df[[Target_column]]),
        columns=region_cols,
        index=train_df.index
    )
    
    # 处理测试集（如果存在）
    if test_df is not None:
        test_df[Target_column] = test_df[Target_column].fillna(0).astype(int).astype(str)
        # 生成编码
        test_encoded = pd.DataFrame(
            encoder.transform(test_df[[Target_column]]),
            columns=region_cols,
            index=test_df.index
        )
        return pd.concat([train_df, train_encoded], axis=1), pd.concat([test_df, test_encoded], axis=1)
    
    return pd.concat([train_df, train_encoded], axis=1)


In [17]:
# 应用处理
Traindata, Testdata = process_region(Traindata, Testdata,'区域')

In [18]:
# 应用处理
Traindata, Testdata = process_region(Traindata, Testdata,'城市')

In [20]:
# 应用处理
Traindata, Testdata = process_region(Traindata, Testdata,'板块')

### 4、处理文字性文本

#### 提取关键词函数

In [25]:
# 定义函数，在多个列中检索是否有关键词：
def extract_text_features(df, keywords=['地铁', '公交','公园', r'高校|大学', '商场', '幼儿园','商场','学区'], Information_columns=['周边配套'],Dummy_Basic_name='设施'):

    # 为每个关键词创建虚拟变量
    for keyword in keywords:
        Target_keyword = keyword.lower().strip()
        col_name = f"{Dummy_Basic_name}_{Target_keyword}"
        col_results = [] 
        # 在多个列中搜索关键词
        for col in Information_columns:
            Text_to_be_checked = df[col].fillna('').str.lower().str.replace('[^\w\s]', '', regex=True)
            # 判断是否含有关键词
            have_key_words=Text_to_be_checked.str.contains(Target_keyword, regex=True, na=False)
            col_results.append(have_key_words)
        # axis=1 表示沿着列的方向拼接
        # .any寻找有any真值，结果保存在新的列向量
        combined = pd.concat(col_results, axis=1).any(axis=1)
        df[col_name] = combined.astype(int)
    
    return df


In [27]:
# 先处理训练集（确定关键词列表）
keywords = [r'地铁', '公交','公园', r'高校|大学', r'商场|广场',r'奥莱|奥特莱斯', '幼儿园','小学','高中','初中','学区','医院']  
Information_columns = ['房屋优势','核心卖点','周边配套','交通出行']

Traindata = extract_text_features(Traindata, keywords, Information_columns,'设施')
# 再用相同关键词处理测试集
Testdata = extract_text_features(Testdata, keywords, Information_columns,'设施')

In [29]:
Traindata.head()

,Unnamed: 0,编号,城市,区域,板块,环线,小区名称,价格,房屋户型,所在楼层,...,设施_公园,设施_高校|大学,设施_商场|广场,设施_奥莱|奥特莱斯,设施_幼儿园,设施_小学,设施_高中,设施_初中,设施_学区,设施_医院
0,0,0,0,79,111,二至三环,人定湖西里,6564200,2室1厅1厨1卫,中楼层 (共5层),...,1,0,0,0,0,0,0,0,0,1
1,1,1,0,43,231,五至六环,龙跃苑四区,4174000,3室1厅1厨1卫,顶层 (共6层),...,1,0,1,0,0,0,0,0,0,1
2,2,2,0,97,54,五至六环,名都园,16310000,4室2厅1厨4卫,底层 (共3层),...,0,0,0,0,0,0,0,0,0,0
3,3,3,0,62,568,三至四环,保利海德公园,2834600,2房间2卫,低楼层 (共10层),...,0,0,0,0,0,0,0,0,0,0
4,4,4,0,62,226,三至四环,京投银泰琨御府,1954000,1房间1卫,中楼层 (共10层),...,0,0,0,0,0,0,0,0,0,0


#### 重要信息虚拟变量

In [31]:
# 先处理训练集（确定关键词列表）
# 注意：此处省去普通住宅的共线性
keywords = ['商业办公', '公寓', '别墅' , '写字楼' ,'新式里弄', '四合院', '商住两用', '底商', '车库']  
Information_columns = ['房屋用途']

Traindata = extract_text_features(Traindata, keywords, Information_columns,'房屋种类')

# 再用相同关键词处理测试集
Testdata = extract_text_features(Testdata, keywords, Information_columns,'房屋种类')

In [32]:
keywords = ['精装', '简装', '毛胚' ]  
Information_columns = ['装修情况']

Traindata = extract_text_features(Traindata, keywords, Information_columns,'装修类型')
Testdata = extract_text_features(Testdata, keywords, Information_columns,'装修类型')

In [33]:
keywords = ['南 北']  
Information_columns = ['房屋朝向']

Traindata = extract_text_features(Traindata, keywords, Information_columns,'朝向')
Testdata = extract_text_features(Testdata, keywords, Information_columns,'朝向')

In [34]:
keywords = ['独栋', '双拼', '叠拼','联排']  
Information_columns = ['别墅类型']

Traindata = extract_text_features(Traindata, keywords, Information_columns,'别墅类型')
Testdata = extract_text_features(Testdata, keywords, Information_columns,'别墅类型')

In [35]:
keywords = ['未满两年', '满五年','满两年']  
Information_columns = ['房屋年限']

Traindata = extract_text_features(Traindata, keywords, Information_columns,'房屋年限')
Testdata = extract_text_features(Testdata, keywords, Information_columns,'房屋年限')

In [36]:
# 注意：此处省去‘商品房’住宅的共线性
keywords = [r'安置房|还建房|拆迁房', r'公房|央产房','集资房','经济适用房']  
Information_columns = ['交易权属']

Traindata = extract_text_features(Traindata, keywords, Information_columns,'交易权属')
Testdata = extract_text_features(Testdata, keywords, Information_columns,'交易权属')

In [37]:
# 先处理训练集（确定关键词列表）
keywords = [r'2024|2025']  
Information_columns = ['交易时间']

Traindata = extract_text_features(Traindata, keywords, Information_columns,'时间')
Testdata = extract_text_features(Testdata, keywords, Information_columns,'时间')

#### 处理房间数据

In [39]:
import pandas as pd
import numpy as np
import re

def get_room_score(df):
    """处理房屋户型，兼容两种格式：'n房间m卫' 和 'n室m厅x厨y卫'"""
    def extract_room(text):
        try:
            text = str(text).strip()
            # 统一将"房间"替换为"室"，简化后续处理
            text = text.replace("房间", "室")

            # 初始化默认值
            room = {'室': 3, '厅': 1, '厨': 1, '卫': 1}
            
            # 正则表达式匹配所有数字+单位的结构（如3室、2厅等）
            pattern = r'(\d+)(室|厅|厨|卫)'
            matches = re.findall(pattern, text)
            
            # 遍历匹配结果，填充数值
            for value, key in matches:
                room[key] = int(value)
            
            return pd.Series([room['室'], room['厅'], room['厨'], room['卫']], index=['室', '厅', '厨', '卫'])
        except:
            # 异常情况返回预设
            return pd.Series([3, 1, 1, 1], index=['室', '厅', '厨', '卫'])
    
    # --- 执行提取 ---
    df[['室', '厅', '厨', '卫']] = df['房屋户型'].apply(extract_room)
    
    # --- 后续处理 ---
    # 按比例计算户型得分（可调整权重）
    df['户型得分'] = df['室']*1 + df['厅']*0.5 + df['厨']*0.3 + df['卫']*0.3
    
    # 生成虚拟变量
    df['有厅'] = np.where(df['厅'] > 0, 1, 0)
    df['有厨'] = np.where(df['厨'] > 0, 1, 0)
    df['有卫'] = np.where(df['卫'] > 0, 1, 0)
    
    # 验证输出
    print("\n【处理结果验证】")
    print(df[['房屋户型', '室', '厅', '厨', '卫', '户型得分']].head(10))
    return df


get_room_score(Traindata)
get_room_score(Testdata)


【处理结果验证】
       房屋户型  室  厅  厨  卫  户型得分
0  2室1厅1厨1卫  2  1  1  1   3.1
1  3室1厅1厨1卫  3  1  1  1   4.1
2  4室2厅1厨4卫  4  2  1  4   6.5
3     2房间2卫  2  1  1  2   3.4
4     1房间1卫  1  1  1  1   2.1
5     1房间1卫  1  1  1  1   2.1
6  3室1厅1厨2卫  3  1  1  2   4.4
7  3室1厅1厨2卫  3  1  1  2   4.4
8  2室1厅1厨1卫  2  1  1  1   3.1
9  5室1厅1厨2卫  5  1  1  2   6.4

【处理结果验证】
       房屋户型  室  厅  厨  卫  户型得分
0  3室2厅1厨3卫  3  2  1  3   5.2
1  4室2厅1厨2卫  4  2  1  2   5.9
2  2室2厅1厨1卫  2  2  1  1   3.6
3  3室2厅1厨2卫  3  2  1  2   4.9
4  2室1厅1厨1卫  2  1  1  1   3.1
5  2室1厅1厨1卫  2  1  1  1   3.1
6  3室2厅1厨2卫  3  2  1  2   4.9
7  2室2厅1厨1卫  2  2  1  1   3.6
8  4室3厅1厨3卫  4  3  1  3   6.7
9  3室1厅1厨1卫  3  1  1  1   4.1


,Unnamed: 0,ID,城市,区域,板块,环线,小区名称,房屋户型,所在楼层,建筑面积,...,交易权属_经济适用房,时间_2024|2025,室,厅,厨,卫,户型得分,有厅,有厨,有卫
0,0,0,0,45,416,四至五环,泛海容郡,3室2厅1厨3卫,中楼层 (共25层),209.20,...,0,1,3,2,1,3,5.2,1,1,1
1,1,1,0,45,414,四至五环,慧谷金色家园,4室2厅1厨2卫,低楼层 (共9层),163.69,...,0,1,4,2,1,2,5.9,1,1,1
2,2,2,0,43,289,五至六环,天通苑中苑,2室2厅1厨1卫,中楼层 (共8层),102.92,...,1,1,2,2,1,1,3.6,1,1,1
3,3,3,0,39,374,NaN,富乐小区南里,3室2厅1厨2卫,中楼层 (共6层),109.66,...,0,1,3,2,1,2,4.9,1,1,1
4,4,4,0,79,724,二至三环,百万庄未区,2室1厅1厨1卫,底层 (共3层),57.20,...,0,1,2,1,1,1,3.1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14781,14781,14781,6,71,244,NaN,横巷市委家属院南区,2室1厅1厨1卫,中楼层 (共7层),65.78,...,0,1,2,1,1,1,3.1,1,1,1
14782,14782,14782,6,91,723,NaN,润天小区,3室2厅1厨2卫,高楼层 (共6层),138.01,...,0,1,3,2,1,2,4.9,1,1,1
14783,14783,14783,6,83,380,三环外,御湖蓝湾,3室2厅1厨2卫,中楼层 (共18层),110.00,...,0,1,3,2,1,2,4.9,1,1,1
14784,14784,14784,6,83,380,三环外,御湖蓝湾,3室2厅1厨2卫,高楼层 (共9层),136.00,...,0,1,3,2,1,2,4.9,1,1,1


#### 处理楼层数据

In [41]:
def calculate_floor(df_floor):
    # 删除历史生成的列（如果存在）
    existing_columns = df_floor.columns
    cols_to_drop = [col for col in existing_columns 
                    if col.startswith(('总层数_', '楼层_','交乘项楼层')) or '_x_' in col]
    df_floor = df_floor.drop(columns=cols_to_drop, errors='ignore')
    
    # 提取总层数和楼层位置
    df_floor['总层数'] = df_floor['所在楼层'].str.extract(r'共(\d+)层').astype(int)
    df_floor['楼层位置'] = df_floor['所在楼层'].str.extract(r'(底|低|中|高|顶)楼层?层?').replace({'顶': '顶层'})
    
    # 分箱总层数
    bins = [0, 4, 9, 19, 29, float('inf')]
    labels = ['房高超低', '房高低层', '房高中层', '房高高层', '房高超高层']
    df_floor['总层数类别'] = pd.cut(df_floor['总层数'], bins=bins, labels=labels, right=True)
    
    # 生成虚拟变量（直接转为int类型）
    dummy_total_floor = pd.get_dummies(df_floor['总层数类别'], prefix='总层数', dtype=int)
    dummy_floor_pos = pd.get_dummies(df_floor['楼层位置'], prefix='楼层', dtype=int)
    
    # 生成交乘项（确保所有组合都存在）
    interaction_terms = pd.DataFrame()
    for t_col in dummy_total_floor.columns:
        for l_col in dummy_floor_pos.columns:
            interaction_terms[f'交乘项楼层_{t_col}_x_{l_col}'] = dummy_total_floor[t_col] * dummy_floor_pos[l_col]
    
    # 合并结果
    df_floor = pd.concat([df_floor, dummy_total_floor, dummy_floor_pos, interaction_terms], axis=1)
    # 结果预览（仅显示新增列）
    # new_columns = list(dummy_total_floor.columns) + list(dummy_floor_pos.columns) + list(interaction_terms.columns)
    # print(df_floor[new_columns].head())
    return df_floor

# 测试运行
Traindata = calculate_floor(Traindata)
Testdata = calculate_floor(Testdata)

### 5、生成交乘项

In [43]:
# 定义函数
def generate_multiply_feature(df,start_name, col_1, col_2, type=1): 
    new_columns = {}
    col_name=df.columns
    # 传入1，为两列相乘
    if type==1:
        new_columns[f'交乘项{start_name}_{col_1}X{col_2}'] = df[col_1]*df[col_2]
        
    # 传入2，搜索以。。。开头列相乘
    elif type==2:    
        for col1 in col_name:
            if col1.startswith(col_1):
                for col2 in col_name:
                    if col2.startswith(col_2):
                        new_columns[f'交乘项{start_name}_{col1}X{col2}'] = df[col1]*df[col2]
    
    # 传入3或其它：col_1乘以col_2开头的列
    else:
        for col2 in col_name:
            if col2.startswith(col_2):
                new_columns[f'交乘项{start_name}_{col_1}X{col2}'] = df[col_1]*df[col2]

    df = pd.concat([df, pd.DataFrame(new_columns)], axis=1)
    return df
        

In [45]:
Traindata = generate_multiply_feature(Traindata, '房租和面积', '平均每平租金(元)', '套内面积', 1)
Testdata = generate_multiply_feature(Testdata, '房租和面积', '平均每平租金(元)', '套内面积', 1)

In [47]:
Traindata = generate_multiply_feature(Traindata, '房租和城市', '平均每平租金(元)', '城市_', type=3)
Testdata = generate_multiply_feature(Testdata, '房租和城市', '平均每平租金(元)', '城市_', type=3)

In [48]:
Traindata = generate_multiply_feature(Traindata, '房租和区域', '平均每平租金(元)', '区域_', type=3)
Testdata = generate_multiply_feature(Testdata, '房租和区域', '平均每平租金(元)', '区域_', type=3)

In [50]:
Traindata = generate_multiply_feature(Traindata, '城市和设施', '城市_', '设施_', type=2)
Testdata = generate_multiply_feature(Testdata, '城市和设施', '城市_', '设施_', type=2)

In [53]:
Traindata = generate_multiply_feature(Traindata, '区域和设施', '区域_', '设施_', type=2)
Testdata = generate_multiply_feature(Testdata, '区域和设施', '区域_', '设施_', type=2)

In [55]:
Traindata = generate_multiply_feature(Traindata, '区域和面积', '套内面积', '区域_', type=2)
Testdata = generate_multiply_feature(Testdata, '区域和面积', '套内面积', '区域_', type=2)

In [57]:
Traindata = generate_multiply_feature(Traindata, '城市和面积', '套内面积', '城市_', type=2)
Testdata = generate_multiply_feature(Testdata, '城市和面积', '套内面积', '城市_', type=2)

In [60]:
Traindata = generate_multiply_feature(Traindata, '城市和别墅类型', '别墅类型_', '城市_', type=2)
Testdata = generate_multiply_feature(Testdata, '城市和别墅类型', '别墅类型_', '城市_', type=2)

In [61]:
Traindata = generate_multiply_feature(Traindata, '区域和别墅类型', '别墅类型_', '区域_', type=2)
Testdata = generate_multiply_feature(Testdata, '区域和别墅类型', '别墅类型_', '区域_', type=2)

In [62]:
Traindata = generate_multiply_feature(Traindata, '城市和建成年限', '建成年限', '城市_', type=3)
Testdata = generate_multiply_feature(Testdata, '城市和建成年限', '建成年限', '城市_', type=3)

In [63]:
Traindata = generate_multiply_feature(Traindata, '城市和供暖', '供暖', '城市_', type=3)
Testdata = generate_multiply_feature(Testdata, '城市和供暖', '供暖', '城市_', type=3)

In [64]:
Traindata = generate_multiply_feature(Traindata, '城市和户型得分', '户型得分', '城市_', type=3)
Testdata = generate_multiply_feature(Testdata, '城市和户型得分', '户型得分', '城市_', type=3)

In [65]:
Traindata = generate_multiply_feature(Traindata, '区域和户型得分', '户型得分', '区域_', type=3)
Testdata = generate_multiply_feature(Testdata, '区域和户型得分', '户型得分', '区域_', type=3)

In [66]:
Traindata = generate_multiply_feature(Traindata, '城市和容积率', '容积率', '城市_', type=3)
Testdata = generate_multiply_feature(Testdata, '城市和容积率', '容积率', '城市_', type=3)

In [67]:
Traindata = generate_multiply_feature(Traindata, '城市和绿化率', '绿化率', '城市_', type=3)
Testdata = generate_multiply_feature(Testdata, '城市和绿化率', '绿化率', '城市_', type=3)

In [68]:
Traindata = generate_multiply_feature(Traindata, '城市和时间', '时间_', '城市_', type=2)
Testdata = generate_multiply_feature(Testdata, '城市和时间', '时间_', '城市_', type=2)

In [69]:
Traindata = generate_multiply_feature(Traindata, '区域和时间', '时间_', '区域_', type=2)
Testdata = generate_multiply_feature(Testdata, '区域和时间', '时间_', '区域_', type=2)

In [70]:
Traindata = generate_multiply_feature(Traindata, '城市和房屋种类', '房屋种类_', '城市_', type=2)
Testdata = generate_multiply_feature(Testdata, '城市和房屋种类', '房屋种类_', '城市_', type=2)

'城市和房屋种类','区域和时间','城市和时间','城市和绿化率','城市和容积率','区域和户型得分','城市和户型得分','城市和供暖','城市和建成年限','区域和别墅类型','城市和别墅类型','区域和设施'

### 6、生成高次方项

In [73]:
def generate_nth_power(df,  column, n=2 ):
    df[f'{n}次方_{column}'] = df[column]**n
    return df
    

In [74]:
Traindata = generate_nth_power(Traindata, '套内面积' , 2)
Testdata = generate_nth_power(Testdata, '套内面积' , 2)
Traindata = generate_nth_power(Traindata, '套内面积' , 3)
Testdata = generate_nth_power(Testdata, '套内面积' , 3)

In [75]:
Traindata = generate_nth_power(Traindata, '平均每平租金(元)' , 2)
Testdata = generate_nth_power(Testdata, '平均每平租金(元)' , 2)
Traindata = generate_nth_power(Traindata, '平均每平租金(元)' , 3)
Testdata = generate_nth_power(Testdata, '平均每平租金(元)' , 3)

In [76]:
Traindata = generate_nth_power(Traindata, '交乘项房租和面积_平均每平租金(元)X套内面积' , 2)
Testdata = generate_nth_power(Testdata, '交乘项房租和面积_平均每平租金(元)X套内面积' , 2)
Traindata = generate_nth_power(Traindata, '交乘项房租和面积_平均每平租金(元)X套内面积' , 3)
Testdata = generate_nth_power(Testdata, '交乘项房租和面积_平均每平租金(元)X套内面积' , 3)

In [77]:
Traindata = generate_nth_power(Traindata, '户型得分' , 2)
Testdata = generate_nth_power(Testdata, '户型得分' , 2)
Traindata = generate_nth_power(Traindata, '户型得分' , 3)
Testdata = generate_nth_power(Testdata, '户型得分' , 3)

### 6、保存数据

In [79]:
Traindata.head()

,Unnamed: 0,编号,城市,区域,板块,环线,小区名称,价格,房屋户型,所在楼层,...,交乘项城市和房屋种类_房屋种类_车库X城市_5,交乘项城市和房屋种类_房屋种类_车库X城市_6,2次方_套内面积,3次方_套内面积,2次方_平均每平租金(元),3次方_平均每平租金(元),2次方_交乘项房租和面积_平均每平租金(元)X套内面积,3次方_交乘项房租和面积_平均每平租金(元)X套内面积,2次方_户型得分,3次方_户型得分
0,0,0,0,79,111,二至三环,人定湖西里,6564200,2室1厅1厨1卫,中楼层 (共5层),...,0.0,0.0,2468.599225,1.226524e+05,20627.070711,2.962486e+06,5.091997e+07,3.633559e+11,9.61,29.791
1,1,1,0,43,231,五至六环,龙跃苑四区,4174000,3室1厅1厨1卫,顶层 (共6层),...,0.0,0.0,15487.304704,1.927364e+06,4268.554218,2.788825e+05,6.610840e+07,5.375080e+11,16.81,68.921
2,2,2,0,97,54,五至六环,名都园,16310000,4室2厅1厨4卫,底层 (共3层),...,0.0,0.0,47138.054769,1.023428e+07,15252.957769,1.883784e+06,7.189948e+08,1.927918e+13,42.25,274.625
3,3,3,0,62,568,三至四环,保利海德公园,2834600,2房间2卫,低楼层 (共10层),...,0.0,0.0,1038.901824,3.348588e+04,54892.790063,1.286095e+07,5.702822e+07,4.306602e+11,11.56,39.304
4,4,4,0,62,226,三至四环,京投银泰琨御府,1954000,1房间1卫,中楼层 (共10层),...,0.0,0.0,1019.014084,3.252897e+04,29753.668008,5.132285e+06,3.031941e+07,1.669479e+11,4.41,9.261


In [80]:
Testdata.to_feather(r'D:\Python_Output\FinalVersion_Testdata.feather')
Traindata.to_feather(r'D:\Python_Output\FinalVersion_Traindata.feather')

# 三、模型拟合：线性linear模型

### 1、读取数据，选择合适的变量作为特征X

##### 读取数据，查看可用列

In [1]:
import pandas as pd
Train = pd.read_feather(r'D:\Python_Output\FinalVersion_Traindata.feather')
Test = pd.read_feather(r'D:\Python_Output\FinalVersion_Testdata.feather')

In [111]:

# 临时设置显示所有列（不影响其他部分的默认行为）
with pd.option_context('display.max_columns', None):
    print("\n【Train可用列】")
    print('\n','='*40,'\n')
    print(Train.columns.tolist())  # 转为列表确保完整显示
    print('\n','='*40,'\n')

    print("\n\n\n【Test可用列】")
    print('\n','='*40,'\n')
    print(Test.columns.tolist())



【Train可用列】


['Unnamed: 0', '编号', '城市', '区域', '板块', '环线', '小区名称', '价格', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向', '建筑结构', '装修情况', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', '年份', '绿化率', '容积率', '供水', '供暖', '建成年限', '平均每平租金(元)', '区域_100', '区域_101', '区域_102', '区域_11', '区域_12', '区域_13', '区域_15', '区域_17', '区域_18', '区域_19', '区域_2', '区域_20', '区域_21', '区域_22', '区域_23', '区域_24', '区域_25', '区域_27', '区域_29', '区域_3', '区域_30', '区域_32', '区域_33', '区域_34', '区域_35', '区域_37', '区域_38', '区域_39', '区域_4', '区域_40', '区域_41', '区域_42', '区域_43', '区域_44', '区域_45', '区域_46', '区域_47', '区域_48', '区域_49', '区域_5', '区域_50', '区域_51', '区域_52', '区域_53', '区域_54', '区域_55', '区域_56', '区域_57', '区域_58', '区域_59', '区域_6', '区域_60', '区域_61', '区域_62', '区域_64', '区域_65', '区域_66', '区域_67', '区域_68', '区域_69', '区域_7', '区域_70', '区域_71', '区域_74', '区域_75', '区域_76', '区域_77', '区域_78', '区域_79', '区域_8', '区域_80', '区域_81', '区域_82', '区域_83', '区域_84', '区域_86', '区域_87', '区域_88', '区域_89', '区域_9',

##### 选取合适的特征

In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# 定义需要匹配的列名前缀列表
# prefix_list = ['城市' ,'板块', '区域', '设施', '房屋种类', '朝向', '别墅类型', '房屋年限', '交易权属', '时间', '装修类型']
prefix_list = ['城市' ,
               '设施', '房屋种类', '朝向', '别墅类型', '房屋年限', '交易权属', '时间', '装修类型',
               '绿化率','容积率', '建成年限','供水','供暖','平均每平租金(元)',
                '户型得分','有厅','有卫','有厨']

'''
prefix_list = ['城市' , '设施', '房屋种类', '朝向', '别墅类型', '房屋年限', '交易权属', '时间', '装修类型',
               '绿化率','容积率', '建成年限','供水','供暖',
               '交乘项房租和面积','交乘项房租和城市_','交乘项城市和设施','交乘项城市和面积',
               '交乘项房租和区域','交乘项区域和面积_',
                '户型得分','有厅','有卫','有厨']
'''
# 获取所有符合前缀条件的列
# exclude_cols = ['区域', '别墅类型', '交易权属', '房屋年限','城市' ,'板块']
exclude_cols = ['别墅类型', '交易权属', '房屋年限','城市','区域','板块']

# 合并选取特征列
feature_cols = [
    col for col in Train.columns 
    if (col.startswith(tuple(prefix_list)) or col in ['配备电梯', '套内面积'])  # 原始选择条件
    # if (col.startswith(tuple(prefix_list)) or col in ['配备电梯', '套内面积'])  # 原始选择条件
    and col not in exclude_cols  # 新增排除条件
]

# 验证列是否存在
missing_cols = set(feature_cols) - set(Train.columns)
if missing_cols:
    print(f"警告：以下特征列不存在，请检查列名拼写: {missing_cols}")
    feature_cols = list(set(feature_cols) - missing_cols)

# 构建特征矩阵和标签
X_train = Train[feature_cols]
y_train = Train['价格']

# 处理测试集（确保列一致）
X_test = Test.reindex(columns=feature_cols, fill_value=0)  # 缺失列填充0

In [64]:
print(f"最终使用的特征列({len(feature_cols)}个):\n", feature_cols)

最终使用的特征列(56个):
 ['套内面积', '配备电梯', '绿化率', '容积率', '供水', '供暖', '建成年限', '平均每平租金(元)', '城市_0', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6', '设施_地铁', '设施_公交', '设施_公园', '设施_高校|大学', '设施_商场|广场', '设施_奥莱|奥特莱斯', '设施_幼儿园', '设施_小学', '设施_高中', '设施_初中', '设施_学区', '设施_医院', '房屋种类_商业办公', '房屋种类_公寓', '房屋种类_别墅', '房屋种类_写字楼', '房屋种类_新式里弄', '房屋种类_四合院', '房屋种类_商住两用', '房屋种类_底商', '房屋种类_车库', '装修类型_精装', '装修类型_简装', '装修类型_毛胚', '朝向_南 北', '别墅类型_独栋', '别墅类型_双拼', '别墅类型_叠拼', '别墅类型_联排', '房屋年限_未满两年', '房屋年限_满五年', '房屋年限_满两年', '交易权属_安置房|还建房|拆迁房', '交易权属_公房|央产房', '交易权属_集资房', '交易权属_经济适用房', '时间_2024|2025', '户型得分', '有厅', '有厨', '有卫']


### 2、进行线性回归

In [60]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

def linear_regression_model(X_train, y_train, X_test):
    # 1、
    print("\n" + "="*40)
    print("线性回归")
    print("="*40)

    # 初始化存储组件
    models = []      # 保存每个折的模型
    scalers = []     # 保存每个折的标准化器
    cv_train_preds = np.zeros(len(X_train))  # 存储交叉验证的训练预测

    # 2、交叉验证训练
    kf = KFold(n_splits=6, shuffle=True, random_state=111)
    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train), 1):
        # 划分数据
        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # 标准化处理（每个折独立标准化）
        scaler = StandardScaler()
        X_fold_train_scaled = scaler.fit_transform(X_fold_train)
        X_fold_val_scaled = scaler.transform(X_fold_val)

        # 模型训练
        model = LinearRegression()
        model.fit(X_fold_train_scaled, y_fold_train)

        # 保存组件
        models.append(model)
        scalers.append(scaler)

        # 记录验证集的预测（用于后续评估）
        val_pred = model.predict(X_fold_val_scaled)
        cv_train_preds[val_idx] = val_pred

        # 打印当前折的验证集表现
        fold_mse = mean_squared_error(y_fold_val, val_pred)
        fold_R2 = r2_score(y_fold_val, val_pred)
        fold_RMSE = np.sqrt(mean_squared_error(y_fold_val, val_pred))
        fold_MAE = mean_absolute_error(y_fold_val, val_pred)
        print(f"Fold {fold_idx} | Val MSE: {fold_mse:.0f}| Val R^2: {fold_R2:.4f}| Val RMSE: {fold_RMSE:.0f}| Val MAE: {fold_MAE:.0f}")
    

    # 3、交叉验证指标
    print("\n=== 交叉验证表现 ===")
    print(f"CV平均MSE: {mean_squared_error(y_train, cv_train_preds):.4f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_train, cv_train_preds)):.4f}")
    print(f"MAE: {mean_absolute_error(y_train, cv_train_preds):.4f}")
    print(f"CV R²: {r2_score(y_train, cv_train_preds):.4f}")

    all_coefs = np.array([model.coef_ for model in models])
    mean_coefs = np.mean(all_coefs, axis=0)
    
    # 创建系数DataFrame并按绝对值排序
    coef_df = pd.DataFrame({
        'Feature': X_train.columns,
        'Coefficient': mean_coefs,
        'Abs_Coefficient': np.abs(mean_coefs)
    }).sort_values(by='Abs_Coefficient', ascending=False)
    
    # 打印系数排名
    print("\n=== 模型参数（按系数绝对值排序） ===")
    print(coef_df[['Feature', 'Coefficient']].to_string(index=False))

    # 4、集成模型预测测试集
    test_preds = []
    for model, scaler in zip(models, scalers):
        X_test_scaled = scaler.transform(X_test)  # 用对应折的标准化器处理测试集
        test_preds.append(model.predict(X_test_scaled))
    
    # 取预测平均值
    y_pred = np.mean(test_preds, axis=0)

    # 5、保存所有子模型及相关组件
    joblib.dump({'models': models, 'scalers': scalers}, 'linear_ensemble_model.pkl')
    print("\n模型已保存为 linear_ensemble_model.pkl")

    # 6、生成预测结果
    predictions = pd.DataFrame({
        'ID': range(len(X_test)),
        'Price': y_pred.round(2)
    })
    return predictions

# 执行训练预测
Prediction = linear_regression_model(X_train, y_train, X_test)
print(Prediction)


线性回归
Fold 1 | Val MSE: 2528073148050| Val R^2: 0.6466| Val RMSE: 1589992| Val MAE: 853738
Fold 2 | Val MSE: 2430985960402| Val R^2: 0.6447| Val RMSE: 1559162| Val MAE: 844773
Fold 3 | Val MSE: 5193834146348| Val R^2: 0.2914| Val RMSE: 2278998| Val MAE: 826695
Fold 4 | Val MSE: 2366743179709| Val R^2: 0.6595| Val RMSE: 1538422| Val MAE: 839473
Fold 5 | Val MSE: 2406506297420| Val R^2: 0.6434| Val RMSE: 1551292| Val MAE: 846936
Fold 6 | Val MSE: 2494672223624| Val R^2: 0.6327| Val RMSE: 1579453| Val MAE: 848640

=== 交叉验证表现 ===
CV平均MSE: 2903464697323.1699
RMSE: 1703955.6031
MAE: 843375.8854
CV R²: 0.5834

=== 模型参数（按系数绝对值排序） ===
         Feature   Coefficient
         装修类型_毛胚 -6.465137e+17
           设施_学区  6.284500e+17
            城市_2 -6.164760e+17
            城市_0 -5.358185e+17
            城市_6 -5.339484e+17
            城市_5 -4.384440e+17
            城市_4 -3.586903e+17
            城市_1 -3.169689e+17
            城市_3 -3.054847e+17
       平均每平租金(元)  1.111463e+06
            户型得分  6.79032

### 3、生成预测结果。返回1调整特征至最优

In [62]:
Prediction.to_csv(r'D:\Python_Output\Lineaner_Prediction.csv', index=None)
print('Saved')

Saved


# 四、Lasso回归

### 1、选取合适的特征作为X （参考三）

In [38]:
'''
prefix_list = ['城市' ,'区域',
               '设施', '房屋种类', '朝向', '别墅类型', '房屋年限', '交易权属', '时间', '装修类型',
               '绿化率','容积率','供水','供暖', '建成年限', '平均每平租金(元)',
               '户型得分','有厅','有卫','有厨',
                '交乘项楼层_','楼层_','总层数_']

prefix_list = ['城市' ,'区域',
               '设施', '房屋种类', '朝向', '别墅类型', '房屋年限', '交易权属', '时间', '装修类型',
               '绿化率','容积率', '建成年限','供水','供暖','平均每平租金(元)',
               '交乘项房租和面积','交乘项房租和城市_','交乘项城市和设施','交乘项城市和面积',
               '交乘项房租和区域','交乘项区域和面积_',
                '户型得分','有厅','有卫','有厨',
              '交乘项楼层_','楼层_','总层数_']
'''

prefix_list = ['城市' ,'区域',
               '设施', '房屋种类', '朝向', '别墅类型', '房屋年限', '交易权属', '时间', '装修类型',
               '绿化率','容积率', '建成年限','供水','供暖','平均每平租金(元)',
               '交乘项房租和面积','交乘项房租和城市_','交乘项城市和设施','交乘项城市和面积',
               '交乘项房租和区域','交乘项区域和面积_',
               
               '交乘项城市和房屋种类','交乘项城市和时间','交乘项城市和绿化率','交乘项城市和容积率',
               '交乘项城市和户型得分','交乘项城市和供暖','交乘项城市和建成年限','交乘项城市和别墅类型',

              '2次方_套内面积','2次方_平均每平租金(元)','2次方_交乘项房租和面积_','2次方_户型得分',
               '3次方_套内面积','3次方_平均每平租金(元)','3次方_交乘项房租和面积_','3次方_户型得分',
                '户型得分','有厅','有卫','有厨',
              '交乘项楼层_','楼层_','总层数_']

exclude_cols = ['板块','别墅类型', '交易权属', '房屋年限','板块','城市','区域']

# 合并选取特征列
feature_cols = [
    col for col in Train.columns 
    if (col.startswith(tuple(prefix_list)) or col in ['配备电梯', '套内面积'])  # 原始选择条件
    # if (col.startswith(tuple(prefix_list)) or col in ['配备电梯', '套内面积'])  # 原始选择条件
    and col not in exclude_cols  # 新增排除条件
]


X_train = Train[feature_cols]
y_train = Train['价格']

X_test = Test.reindex(columns=feature_cols, fill_value=0)  # 缺失列填充0

print(f"最终使用的特征列({len(feature_cols)}个):\n", feature_cols)

最终使用的特征列(50个):
 ['套内面积', '配备电梯', '绿化率', '容积率', '建成年限', '城市_0', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6', '设施_地铁', '设施_公交', '设施_公园', '设施_高校|大学', '设施_商场|广场', '设施_奥莱|奥特莱斯', '设施_幼儿园', '设施_小学', '设施_高中', '设施_初中', '设施_学区', '设施_医院', '房屋种类_商业办公', '房屋种类_公寓', '房屋种类_别墅', '房屋种类_写字楼', '房屋种类_新式里弄', '房屋种类_四合院', '房屋种类_商住两用', '房屋种类_底商', '房屋种类_车库', '装修类型_精装', '装修类型_简装', '装修类型_毛胚', '朝向_南 北', '别墅类型_独栋', '别墅类型_双拼', '别墅类型_叠拼', '别墅类型_联排', '房屋年限_未满两年', '房屋年限_满五年', '房屋年限_满两年', '交易权属_安置房|还建房|拆迁房', '交易权属_公房|央产房', '交易权属_集资房', '交易权属_经济适用房', '时间_2024|2025', '户型得分']


### 2、训练Lasso模型

In [81]:
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

def lasso_regression_model(X_train, y_train, X_test):
    print("\n" + "="*40)
    print("Lasso回归训练")
    print("="*40)
 
    # 1、通过网格搜索确定最佳alpha

    temp_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('model', Lasso(max_iter=10000))
    ])
    '''
    # 参数网格搜索
    param_grid = {'model__alpha': np.logspace(-4, 2, 20)}
    grid = GridSearchCV(
        temp_pipe,
        param_grid,
        scoring='neg_mean_squared_error',
        cv=6,
        n_jobs=-1
    )
    grid.fit(X_train, y_train)
    best_alpha = grid.best_params_['model__alpha']
    print(f"最佳alpha值: {best_alpha:.4f}")
    '''
    best_alpha = 0.01
    # 2、使用最佳alpha进行交叉验证集成
    models = []      # 存储子模型
    scalers = []     # 存储标准化器
    cv_train_preds = np.zeros(len(X_train))  # 存储交叉验证预测
    fold_r2_scores = []  # 记录每个Fold的R2分数
    
    kf = KFold(n_splits=6, shuffle=True, random_state=111)
    # enumerate()用于创建索引，1表示从1开始计数
    # train_idx, val_idx分别对应交叉验证分组后的训练集和测试集
    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train), 1):
        # 数据划分
        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # 标准化处理（每个折独立进行）
        # 注意：我们在训练集上使用.fit_transform，而在测试集上使用.transform，保证模型预测评估由训练集真实得到
        scaler = StandardScaler()
        X_fold_train_scaled = scaler.fit_transform(X_fold_train)
        X_fold_val_scaled = scaler.transform(X_fold_val)

        # 模型训练
        # 注：此处可修改为自定的alpha
        model = Lasso(alpha=best_alpha, max_iter=3000)
        model.fit(X_fold_train_scaled, y_fold_train)
        
        # 存储组件
        models.append(model)
        scalers.append(scaler)
        
        # 记录验证集预测
        val_pred = model.predict(X_fold_val_scaled)
        cv_train_preds[val_idx] = val_pred
        
        # 打印当前折表现
        fold_mse = mean_squared_error(y_fold_val, val_pred)
        fold_R2 = r2_score(y_fold_val, val_pred)
        fold_RMSE = np.sqrt(mean_squared_error(y_fold_val, val_pred))
        fold_MAE = mean_absolute_error(y_fold_val, val_pred)
        print(f"Fold {fold_idx} | Val MSE: {fold_mse:.0f}| Val R^2: {fold_R2:.4f}| Val RMSE: {fold_RMSE:.0f}| Val MAE: {fold_MAE:.0f}")
                # 记录当前Fold的R²
        fold_R2 = r2_score(y_fold_val, val_pred)
        fold_r2_scores.append(fold_R2)  # 保存R²分数

    

    # 3、输出模型表现 
    print("\n=== 交叉验证表现 ===")
    print(f"CV平均MSE: {mean_squared_error(y_train, cv_train_preds):.4f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_train, cv_train_preds)):.4f}")
    print(f"MAE: {mean_absolute_error(y_train, cv_train_preds):.4f}")
    print(f"CV R²: {r2_score(y_train, cv_train_preds):.4f}")

    # 4、输出特征的相关系数
    # 计算平均系数； 注意array为行合并，故axis=0
    all_coefs = np.array([model.coef_ for model in models])
    mean_coefs = np.mean(all_coefs, axis=0)
    
    # 排序
    coef_df = pd.DataFrame({
        '特征': X_train.columns,
        '系数': mean_coefs,
        '系数绝对值': np.abs(mean_coefs)
    }).sort_values('系数绝对值', ascending=False)
    
    print("\n 特征重要性（按系数绝对值排序）：")
    print(coef_df[['特征', '系数']].to_string(index=False))
    
    # 5、预测
    '''
    X_test_aligned = X_test.reindex(columns=X_train.columns, fill_value=0)
    test_preds = []
    
    for model, scaler in zip(models, scalers):
        X_test_scaled = scaler.transform(X_test_aligned)
        test_preds.append(model.predict(X_test_scaled))
    
    y_pred = np.mean(test_preds, axis=0)
    '''
    X_test_aligned = X_test.reindex(columns=X_train.columns, fill_value=0)
    # 计算权重
    fold_weights = np.array(fold_r2_scores)
    fold_weights[fold_weights < 0] = 0  # 负R²的Fold权重归零
    fold_weights /= fold_weights.sum()  # 归一化权重
    # 加权预测
    y_pred = np.zeros(len(X_test_aligned))
    for (model, scaler), weight in zip(zip(models, scalers), fold_weights):
        X_test_scaled = scaler.transform(X_test_aligned)
        y_pred += weight * model.predict(X_test_scaled)
    
    # 7、保存模型
    joblib.dump({'models': models, 'scalers': scalers}, 'lasso_ensemble_model.pkl')
    print("\n模型已保存为 lasso_ensemble_model.pkl")
    # 若要再次使用，modekLasso = joblib.load('lasso_ensemble_model.pkl')

    # 8、合并预测结果
    predictions = pd.DataFrame({
        'ID': range(len(X_test_aligned)),
        'Price': y_pred.round(2)
    })
    return predictions

# 执行示例
results_Lasso = lasso_regression_model(X_train, y_train, X_test)
print(results_Lasso)


Lasso回归训练


D:\Program_File\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.972e+16, tolerance: 4.860e+13
  model = cd_fast.enet_coordinate_descent(


Fold 1 | Val MSE: 599444173046| Val R^2: 0.9162| Val RMSE: 774238| Val MAE: 367289


D:\Program_File\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.961e+16, tolerance: 4.904e+13
  model = cd_fast.enet_coordinate_descent(


Fold 2 | Val MSE: 632499200892| Val R^2: 0.9076| Val RMSE: 795298| Val MAE: 376157


...

Fold 4 | Val MSE: 515117290857| Val R^2: 0.9259| Val RMSE: 717717| Val MAE: 366421


D:\Program_File\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.954e+16, tolerance: 4.917e+13
  model = cd_fast.enet_coordinate_descent(


Fold 5 | Val MSE: 664105039702| Val R^2: 0.9016| Val RMSE: 814926| Val MAE: 371233


D:\Program_File\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.941e+16, tolerance: 4.911e+13
  model = cd_fast.enet_coordinate_descent(


Fold 6 | Val MSE: 711212913404| Val R^2: 0.8953| Val RMSE: 843334| Val MAE: 376486

=== 交叉验证表现 ===
CV平均MSE: 817179343911.4779
RMSE: 903979.7254
MAE: 373082.8896
CV R²: 0.8827

 特征重要性（按系数绝对值排序）：
                         特征            系数
                   3次方_套内面积  1.510795e+06
                   2次方_套内面积 -1.411563e+06
3次方_交乘项房租和面积_平均每平租金(元)X套内面积 -1.032298e+06
                      区域_45 -9.840089e+05
    交乘项房租和城市_平均每平租金(元)X城市_0  9.762256e+05
                  平均每平租金(元)  8.260461e+05
2次方_交乘项房租和面积_平均每平租金(元)X套内面积  8.158688e+05
    交乘项房租和面积_平均每平租金(元)X套内面积  7.730235e+05
                       套内面积  7.458928e+05
              2次方_平均每平租金(元) -7.277333e+05
    交乘项房租和城市_平均每平租金(元)X城市_3  6.875793e+05
                      区域_61 -6.237758e+05
        交乘项区域和面积_套内面积X区域_62  5.909530e+05
       交乘项城市和建成年限_建成年限X城市_0 -5.340832e+05
                      区域_62 -5.320873e+05
                   2次方_户型得分  5.313257e+05
    交乘项城市和房屋种类_房屋种类_公寓X城市_0 -5.184799e+05
        交乘项区域和面积_套内面积X区域_45  5.048196e+05
        

### 3、保存预测结果，重新测评，调整参数

In [83]:
results_Lasso.to_csv(r'D:\Python_Output\Lasso_Predictions.csv', index=None)
print('Saved')

Saved


# 五、Ridge回归

### 1、选择需要的特征

In [67]:
'''
prefix_list = ['城市' ,'区域',
               '设施', '房屋种类', '朝向', '别墅类型', '房屋年限', '交易权属', '时间', '装修类型',
               '绿化率','容积率','供水','供暖', '建成年限', '平均每平租金(元)',
               '户型得分','有厅','有卫','有厨',
                '交乘项楼层_','楼层_','总层数_']

prefix_list = ['城市' ,'区域',
               '设施', '房屋种类', '朝向', '别墅类型', '房屋年限', '交易权属', '时间', '装修类型',
               '绿化率','容积率', '建成年限','供水','供暖','平均每平租金(元)',
               '交乘项房租和面积','交乘项房租和城市_','交乘项城市和设施','交乘项城市和面积',
               '交乘项房租和区域','交乘项区域和面积_',
                '户型得分','有厅','有卫','有厨',
              '交乘项楼层_','楼层_','总层数_']
'''

prefix_list = ['城市' ,'区域',
               '设施', '房屋种类', '朝向', '别墅类型', '房屋年限', '交易权属', '时间', '装修类型',
               '绿化率','容积率', '建成年限','供水','供暖','平均每平租金(元)',
               '交乘项房租和面积','交乘项房租和城市_','交乘项城市和设施','交乘项城市和面积',
               '交乘项房租和区域','交乘项区域和面积_',
               
               '交乘项城市和房屋种类','交乘项城市和时间','交乘项城市和绿化率','交乘项城市和容积率',
               '交乘项城市和户型得分','交乘项城市和供暖','交乘项城市和建成年限','交乘项城市和别墅类型',

              '2次方_套内面积','2次方_平均每平租金(元)','2次方_交乘项房租和面积_','2次方_户型得分',
               '3次方_套内面积','3次方_平均每平租金(元)','3次方_交乘项房租和面积_','3次方_户型得分',
                '户型得分','有厅','有卫','有厨',
              '交乘项楼层_','楼层_','总层数_']

exclude_cols = ['板块','别墅类型', '交易权属', '房屋年限','板块','城市','区域']

# 合并选取特征列
feature_cols = [
    col for col in Train.columns 
    if (col.startswith(tuple(prefix_list)) or col in ['配备电梯', '套内面积'])  # 原始选择条件
    # if (col.startswith(tuple(prefix_list)) or col in ['配备电梯', '套内面积'])  # 原始选择条件
    and col not in exclude_cols  # 新增排除条件
]


X_train = Train[feature_cols]
y_train = Train['价格']

X_test = Test.reindex(columns=feature_cols, fill_value=0)  # 缺失列填充0

print(f"最终使用的特征列({len(feature_cols)}个):\n", feature_cols)

最终使用的特征列(589个):
 ['套内面积', '配备电梯', '绿化率', '容积率', '供水', '供暖', '建成年限', '平均每平租金(元)', '区域_100', '区域_101', '区域_102', '区域_11', '区域_12', '区域_13', '区域_15', '区域_17', '区域_18', '区域_19', '区域_2', '区域_20', '区域_21', '区域_22', '区域_23', '区域_24', '区域_25', '区域_27', '区域_29', '区域_3', '区域_30', '区域_32', '区域_33', '区域_34', '区域_35', '区域_37', '区域_38', '区域_39', '区域_4', '区域_40', '区域_41', '区域_42', '区域_43', '区域_44', '区域_45', '区域_46', '区域_47', '区域_48', '区域_49', '区域_5', '区域_50', '区域_51', '区域_52', '区域_53', '区域_54', '区域_55', '区域_56', '区域_57', '区域_58', '区域_59', '区域_6', '区域_60', '区域_61', '区域_62', '区域_64', '区域_65', '区域_66', '区域_67', '区域_68', '区域_69', '区域_7', '区域_70', '区域_71', '区域_74', '区域_75', '区域_76', '区域_77', '区域_78', '区域_79', '区域_8', '区域_80', '区域_81', '区域_82', '区域_83', '区域_84', '区域_86', '区域_87', '区域_88', '区域_89', '区域_9', '区域_90', '区域_91', '区域_92', '区域_93', '区域_94', '区域_95', '区域_96', '区域_97', '区域_98', '区域_99', '城市_0', '城市_1', '城市_2', '城市_3', '城市_4', '城市_5', '城市_6', '设施_地铁', '设施_公交', '设施_公园', '设施_高校|大学', '设施_商场|广场', '设施_奥莱|

In [69]:
'''
# 示例：剔除目标变量Y的极端值（假设y_train是Series）
y_quantile_low = y_train.quantile(0.05)
y_quantile_high = y_train.quantile(0.95)
clean_mask = (y_train >= y_quantile_low) & (y_train <= y_quantile_high)

X_train_clean = X_train[clean_mask]
y_train_clean = y_train[clean_mask]
'''

'\n# 示例：剔除目标变量Y的极端值（假设y_train是Series）\ny_quantile_low = y_train.quantile(0.05)\ny_quantile_high = y_train.quantile(0.95)\nclean_mask = (y_train >= y_quantile_low) & (y_train <= y_quantile_high)\n\nX_train_clean = X_train[clean_mask]\ny_train_clean = y_train[clean_mask]\n'

### 2、Ridge回归

In [72]:
import numpy as np
np.logspace(-3, 3, 20)

array([1.00000000e-03, 2.06913808e-03, 4.28133240e-03, 8.85866790e-03,
       1.83298071e-02, 3.79269019e-02, 7.84759970e-02, 1.62377674e-01,
       3.35981829e-01, 6.95192796e-01, 1.43844989e+00, 2.97635144e+00,
       6.15848211e+00, 1.27427499e+01, 2.63665090e+01, 5.45559478e+01,
       1.12883789e+02, 2.33572147e+02, 4.83293024e+02, 1.00000000e+03])

In [74]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

def ridge_regression_model(X_train, y_train, X_test):
    print("\n" + "="*40)
    print("Ridge回归训练（交叉验证集成版）")
    print("="*40)

    # 1、寻找合适的alpha参数
    # 临时使用Pipeline仅用于参数搜索
    temp_pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('model', Ridge(max_iter=10000, random_state=111))
    ])
    # np.logspace生成20份e-3到e3, alpha在其中枚举
    param_grid = {'model__alpha': np.logspace(-3, 3, 100)}
    grid = GridSearchCV(
        temp_pipe,
        param_grid,
        scoring='neg_mean_squared_error',
        cv=6,
        # n_jobs: 并行数，int：个数,-1：跟CPU核数一致, 1:默认值
        n_jobs=-1,
        verbose=1
    )
    grid.fit(X_train, y_train)
    best_alpha = grid.best_params_['model__alpha']
    print(f"\n最佳alpha值: {best_alpha:.4f}")

    # 2、使用最佳alpha进行交叉验证集成 
    models = []      # 存储子模型
    scalers = []     # 存储标准化器
    cv_train_preds = np.zeros(len(X_train))  # 存储交叉验证预测
    fold_r2_scores = []  # 记录每个Fold的R2分数
    
    kf = KFold(n_splits=6, shuffle=True, random_state=111)
    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train), 1):
        # 数据划分
        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # 标准化处理（每个折独立进行）
        scaler = StandardScaler()
        X_fold_train_scaled = scaler.fit_transform(X_fold_train)
        X_fold_val_scaled = scaler.transform(X_fold_val)

        # 模型训练
        model = Ridge(alpha=best_alpha, max_iter=10000, random_state=111)
        model.fit(X_fold_train_scaled, y_fold_train)
        
        # 存储组件
        models.append(model)
        scalers.append(scaler)
        
        # 记录验证集预测
        val_pred = model.predict(X_fold_val_scaled)
        cv_train_preds[val_idx] = val_pred
        
        # 打印当前折表现
        fold_mse = mean_squared_error(y_fold_val, val_pred)
        fold_R2 = r2_score(y_fold_val, val_pred)
        fold_RMSE = np.sqrt(mean_squared_error(y_fold_val, val_pred))
        fold_MAE = mean_absolute_error(y_fold_val, val_pred)
        print(f"Fold {fold_idx} | Val MSE: {fold_mse:.0f}| Val R^2: {fold_R2:.4f}| Val RMSE: {fold_RMSE:.0f}| Val MAE: {fold_MAE:.0f}")
        # 记录当前Fold的R²
        fold_R2 = r2_score(y_fold_val, val_pred)
        fold_r2_scores.append(fold_R2)  # 保存R²分数

    # 3、模型的综合表现
    print("\n=== 交叉验证表现 ===")
    print(f"CV平均MSE: {mean_squared_error(y_train, cv_train_preds):.4f}")
    print(f"CV R²: {r2_score(y_train, cv_train_preds):.4f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_train, cv_train_preds)):.4f}")
    print(f"MAE: {mean_absolute_error(y_train, cv_train_preds):.4f}")

    # 4、展示特征的相关系数
    all_coefs = np.array([model.coef_ for model in models])
    mean_coefs = np.mean(all_coefs, axis=0)
    
    coef_df = pd.DataFrame({
        '特征': X_train.columns,
        '系数': mean_coefs,
        '绝对值': np.abs(mean_coefs)
    }).sort_values('绝对值', ascending=False)
    
    print("\n=== 特征重要性（按系数绝对值排序） ===")
    print(coef_df[['特征', '系数']].to_string(index=False, float_format="%.4f"))

    # 5、预测
    '''
    X_test_aligned = X_test.reindex(columns=X_train.columns, fill_value=0)
    test_preds = []
    
    for model, scaler in zip(models, scalers):
        X_test_scaled = scaler.transform(X_test_aligned)
        test_preds.append(model.predict(X_test_scaled))
    
    y_pred = np.mean(test_preds, axis=0)
    '''
    X_test_aligned = X_test.reindex(columns=X_train.columns, fill_value=0)
    # 计算权重
    fold_weights = np.array(fold_r2_scores)
    fold_weights[fold_weights < 0] = 0  # 负R²的Fold权重归零
    fold_weights /= fold_weights.sum()  # 归一化权重
    # 加权预测
    y_pred = np.zeros(len(X_test_aligned))
    for (model, scaler), weight in zip(zip(models, scalers), fold_weights):
        X_test_scaled = scaler.transform(X_test_aligned)
        y_pred += weight * model.predict(X_test_scaled)
    
    # 6、保存模型
    joblib.dump({'models': models, 'scalers': scalers}, 'ridge_ensemble_model.pkl')
    print("\n模型已保存为 ridge_ensemble_model.pkl")

    # 7、合并预测结果
    predictions = pd.DataFrame({
        'ID': X_test_aligned.index,
        'Price': y_pred.round(2)
    })
    return predictions

# 执行示例
results_Ridge = ridge_regression_model(X_train, y_train, X_test)
print(results_Ridge)


Ridge回归训练（交叉验证集成版）
Fitting 6 folds for each of 100 candidates, totalling 600 fits


D:\Program_File\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
81 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "D:\Program_File\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\Program_File\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Program_File\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_tra


最佳alpha值: 756.4633
Fold 1 | Val MSE: 604879419296| Val R^2: 0.9154| Val RMSE: 777740| Val MAE: 368499
Fold 2 | Val MSE: 639538574919| Val R^2: 0.9065| Val RMSE: 799712| Val MAE: 377074
Fold 3 | Val MSE: 8742710129046| Val R^2: -0.1928| Val RMSE: 2956807| Val MAE: 398046
Fold 4 | Val MSE: 519284214285| Val R^2: 0.9253| Val RMSE: 720614| Val MAE: 368231
Fold 5 | Val MSE: 676768838810| Val R^2: 0.8997| Val RMSE: 822660| Val MAE: 372872
Fold 6 | Val MSE: 752041919547| Val R^2: 0.8893| Val RMSE: 867204| Val MAE: 375961

=== 交叉验证表现 ===
CV平均MSE: 1989187395316.4473
CV R²: 0.7146
RMSE: 1410385.5485
MAE: 376780.4050

=== 特征重要性（按系数绝对值排序） ===
                         特征           系数
    交乘项房租和面积_平均每平租金(元)X套内面积  965574.6080
        交乘项区域和面积_套内面积X区域_62  501508.0129
         交乘项城市和面积_套内面积X城市_0  452075.6449
         交乘项城市和面积_套内面积X城市_3  442671.9891
        交乘项区域和面积_套内面积X区域_45  387473.2464
                      区域_45 -382104.8023
                      区域_61 -363351.2843
3次方_交乘项房租和面积_平均每平租金(元)X套内面积 -326

### 3、保存数据

In [76]:
results_Ridge.to_csv(r'D:\Python_Output\Ridge_Prediction.csv', index=None)
print('Saved')

Saved


分点：
一、总体工作内容和流程：
二、数据处理：
1、数据合并（合并技巧）
2、数据处理，介绍用到的特征工程方法，以及优点
优点：
1、函数化，模块化处理：一次项：数值型，文字搜索类型；文字型虚拟变量；方便调用和修改和补充，也确保了格式的统一以及不会犯错；且函数泛用性高
2、特征选取上，基本运用了所有的有效信息：三个数据集，（第二页）

三、模型设定：
1、三种模型；基本流程，模型会做什么：输出
2、模型的改进（此处用流程图展示）：一些问题的解决

（第二页）
解决的问题：1、欠拟合，加入新的特征，以及交乘法项，
2、过拟合：Lasso和Ridge解决；输出参数，删除不合预期的项
3、Lasso没有解析解，需要迭代，收敛速度：设定更大的alpha，改用多线程
4、交叉验证某集合上效果很差：在交叉验证上的改进，数据清洗（无效，拟合降低），采用比例合成最终结果；

模型结果展示
模型优点陈述